In [4]:
%%bash
c++ -g -o _residual.dylib residual.cpp -fPIC -shared \
    -L$HOME/miniconda3/envs/iebeam/lib -I$HOME/miniconda3/envs/iebeam/include;
    
install_name_tool -add_rpath $HOME/miniconda3/envs/iebeam/lib _residual.dylib

In [5]:
import numpy as np
import scipy


import cffi
from scipy.special import legendre
from scipy.optimize import newton_krylov, fsolve, anderson


ffi = cffi.FFI()


ffi.cdef("""
            void computeResidual(const double youngs_mod, const double area, const double moment_of_inertia, 
                       const int int_rule, const double* int_points, const double* int_wts, 
                       const int num_elements, const double* nodes, const double* uknowns, 
                       double* residual);
            """)

res = ffi.dlopen("_residual.dylib")

In [30]:
class InextensibleBeamFEA(object):
    
    def __init__(self, number_of_elements=1, length=1, E=1, I=1, A=1, P2=0.01, int_rule=3):
    
        self.number_of_elements = number_of_elements
        self.length = length
        
        self.E = E
        self.I = I
        self.A = A
        
        self.P2 = P2
        
        self.int_rule = int_rule
        self.xi, self.int_wts = np.polynomial.legendre.leggauss(int_rule)
        
        self.nodes = np.linspace(0, length, num=(number_of_elements * 2 + 1))
        
        number_of_midpoint_nodes = (self.nodes.shape[0]) // 2
        self.unknowns = np.zeros((number_of_midpoint_nodes + 1) * 4 + number_of_midpoint_nodes)
        self.residual = np.zeros_like(self.unknowns)
        
    def compute_residual(self, X):
        
        #Compute the residual from C library
        res.computeResidual(self.E , self.A, self.I, self.int_rule, ffi.from_buffer(self.xi), 
                            ffi.from_buffer(self.int_wts), self.number_of_elements, ffi.from_buffer(self.nodes), 
                            ffi.from_buffer(X), ffi.from_buffer(self.residual))
        
        #Zero out boundary conditions in residual (cantileaver beam)
        #self.residual[0:3] = 0.0
        
        #Apply load to end of beam
        self.residual[-3] -= self.P2
        
        return self.residual
    
    
    def solve(self):
        
        guess = np.zeros_like(self.unknowns)
        
        return newton_krylov(self.compute_residual, guess, verbose=True)

In [31]:
problem = InextensibleBeamFEA(number_of_elements=2, E=200e9)

problem.solve()

0:  |F(x)| = 1.5985e+06; step 1; tol 0.9999
1:  |F(x)| = 2.3881e+06; step 1; tol 0.9999
2:  |F(x)| = 3.53245e+06; step 1; tol 0.9999
3:  |F(x)| = 4.7834e+06; step 1; tol 0.9999
4:  |F(x)| = 6.07546e+06; step 1; tol 0.9999
5:  |F(x)| = 7.3871e+06; step 1; tol 0.9999
6:  |F(x)| = 8.70948e+06; step 1; tol 0.9999
7:  |F(x)| = 1.00384e+07; step 1; tol 0.9999
8:  |F(x)| = 1.13715e+07; step 1; tol 0.9999
9:  |F(x)| = 1.27075e+07; step 1; tol 0.9999
10:  |F(x)| = 1.40457e+07; step 1; tol 0.9999
11:  |F(x)| = 1.53853e+07; step 1; tol 0.9999
12:  |F(x)| = 1.67262e+07; step 1; tol 0.9999
13:  |F(x)| = 1.80679e+07; step 1; tol 0.9999
14:  |F(x)| = 1.94104e+07; step 1; tol 0.9999
15:  |F(x)| = 2.07535e+07; step 1; tol 0.9999
16:  |F(x)| = 2.20971e+07; step 1; tol 0.9999
17:  |F(x)| = 2.34411e+07; step 1; tol 0.9999
18:  |F(x)| = 2.47854e+07; step 1; tol 0.9999
19:  |F(x)| = 2.613e+07; step 1; tol 0.9999
20:  |F(x)| = 2.74749e+07; step 1; tol 0.995028
21:  |F(x)| = 2.882e+07; step 1; tol 0.990281
22

NoConvergence: [  1.42236222e-17   5.38572319e-10   2.10462031e-09   4.78964607e-28
  -1.41918880e-16  -9.83450966e-17  -7.39526051e-09   6.65429044e-09
  -5.10269100e-27   4.25398247e-17   1.83500529e-16  -5.23543147e-09
   3.87732100e-09  -3.59163840e-27]